# Error analysis with gender

In [1]:
# Import necessary libraries
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, auc, precision_recall_curve, confusion_matrix
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import confusion_matrix 
from collections import Counter

In [7]:
# Load the data into pandas dataframes
X_train_res = pd.read_csv('/Users/dionnespaltman/Desktop/V4/data/X_train_res.csv')
y_train_res = pd.read_csv('/Users/dionnespaltman/Desktop/V4/data/y_train_res.csv')
X_test = pd.read_csv('/Users/dionnespaltman/Desktop/V4/data/X_test.csv')
y_test = pd.read_csv('/Users/dionnespaltman/Desktop/V4/data/y_test.csv')

X_train_res = X_train_res.drop(columns='Unnamed: 0', axis=1)
y_train_res = y_train_res.drop(columns='Unnamed: 0', axis=1)
X_test = X_test.drop(columns='Unnamed: 0', axis=1)
y_test = y_test.drop(columns='Unnamed: 0', axis=1)

# # Convert the DataFrame to a NumPy array
# # y_test = y_test.to_numpy()

# # Display the first few rows of each dataframe to verify the loading
# print("X_train_res:")
# display(X_train_res.head())

# print("\ny_train_res:")
# display(y_train_res.head())

# print("\nX_test:")
# display(X_test.head())

# print("\ny_test:")
# # print(y_test)
# display(y_test.head())

# Convert the DataFrame to a 1-dimensional NumPy array
y_train_res = y_train_res.values.ravel()
y_test = y_test.values.ravel()

In [8]:
X_test_gender = pd.read_csv('/Users/dionnespaltman/Desktop/V4/data/X_test_gender.csv')
X_test_gender = X_test_gender.drop(columns='Unnamed: 0', axis=1)
display(X_test_gender)

,VVR_1,VVR_2,Gender,AU01_r__sum_values,AU01_r__variance,AU01_r__standard_deviation,AU01_r__maximum,AU01_r__mean,AU01_r__root_mean_square,AU02_r__sum_values,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,10.0,11.0,1,-1.088267,-0.486929,-0.245322,-0.094909,-0.387079,-0.373556,-0.296967,...,-1.106604,-1.661648,-0.939358,-1.258986,-0.862599,0.657150,0.649175,0.619766,0.540339,0.684272
1,19.0,21.0,2,0.160204,0.681681,0.725849,0.514376,0.751085,0.775827,0.031562,...,1.791290,0.458331,1.507769,1.887518,-0.111414,0.662918,0.653014,0.478080,0.552052,0.691181
2,8.0,8.0,1,-0.510947,-0.678086,-0.451146,-0.130749,-0.740730,-0.650167,-0.512662,...,0.412202,0.458331,-0.395816,0.012935,0.703933,1.133047,0.952462,0.619766,0.765365,1.028129
3,21.0,21.0,2,1.139765,0.115481,0.300321,0.406855,0.152945,0.239720,0.621775,...,-0.009577,0.017708,-0.227168,-0.190216,1.476715,0.191834,0.323467,0.493823,0.412243,0.350324
4,11.0,8.0,1,1.257027,1.622690,1.325337,0.514376,1.084273,1.365502,1.155021,...,0.075374,0.217235,-0.256234,-0.149869,-0.194813,-0.655535,-0.383937,-0.104406,-0.685164,-0.617897
5,10.0,12.0,1,0.790449,-0.100140,0.118978,0.514376,0.073629,0.068108,0.478215,...,1.295683,0.458331,1.426896,1.528188,0.458794,-0.476317,-0.217535,0.273423,-0.421622,-0.388784
6,13.0,12.0,2,0.281915,-1.509396,-2.338926,-2.794178,-0.240768,-1.408717,0.306897,...,-2.446864,-3.087456,-0.232253,-1.135262,0.525877,-1.865693,-2.711519,-2.908383,-0.124630,-1.701399
7,8.0,8.0,1,-0.396322,-0.317742,-0.078736,-0.066237,-0.373000,-0.232622,-0.356449,...,-0.032940,0.458331,-0.212612,-0.197559,-0.254951,-0.082273,0.114081,-0.206735,-0.252523,-0.035261
8,8.0,8.0,1,-0.420808,-0.184904,0.043912,0.514376,-0.491109,-0.163572,0.109287,...,0.228005,0.366881,0.011080,0.088781,-0.313739,-0.526160,-0.262678,-0.175249,-0.497727,-0.452317
9,16.0,13.0,2,1.025705,2.194399,1.645752,0.363847,1.811794,1.860773,-0.347530,...,1.824795,0.458331,2.279075,2.327687,0.912335,1.767250,1.321341,0.415109,2.270614,1.817769


In [ ]:
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=0)

# Create the pipeline with RFE and the model
# rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
rfe = RFE(estimator=DecisionTreeClassifier())
model = SVC(probability=True)
pipeline = Pipeline(steps=[('s', rfe), ('m', model)])

param_grid = {
    # 'm__C': [0.1, 1, 10, 100],  # Regularization parameter
    # 'm__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
    # 'm__gamma': ['scale', 'auto'],  # Kernel coefficient
    # 'm__class_weight': [{0: 0.6428571428571429, 1: 2.25}, None],  # Class weights
    # 's__n_features_to_select': [5, 10, 20, 40, 60, 80]  # Number of features to select with RFE
    'm__C': [0.1],  # Regularization parameter
    'm__kernel': ['poly'],  # Kernel type
    'm__gamma': ['auto'],  # Kernel coefficient
    'm__class_weight': [{0: 0.6428571428571429, 1: 2.25}],  # Class weights
    's__n_features_to_select': [10]  # Number of features to select with RFE
}

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=2
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, X_train_res, y_train_res, cv=outer_cv, n_jobs=2)
print(
    "The mean score using nested cross-validation is: "
    f"{test_score.mean():.3f} ± {test_score.std():.3f}"
)

# Print shape
print(X_train_res.shape)
print(y_train_res.shape)

# Fit model to training data to get best parameters
model.fit(X_train_res, y_train_res)

# Print best parameters
print(model.best_params_)

# Evaluate on the test set
best_model = model.best_estimator_
y_pred = best_model.predict(X_test)

# Print shape
print(y_test.shape)
print(y_pred.shape)

# Print classification report 
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

# Calculate and print the AUC-PR
y_proba = best_model.predict_proba(X_test)[:, 1]  # Get the probabilities for the positive class
precision, recall, _ = precision_recall_curve(y_test, y_proba)
auc_pr = auc(recall, precision)
print(f"AUC-PR: {auc_pr:.3f}")


# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)
# print(metrics.auc(fpr, tpr))